![Ejemplo 2](images/Ej2_exp_trans.png)

In [ ]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Leemos los datos de las corrientes calientes

In [ ]:
Hot_df = pd.read_excel('Datos/Ejercicio2.xlsx', 'Hot',skiprows = 1,index_col=0)
Hot_df

E igualmente de las frías.

In [ ]:
Cold_df = pd.read_excel('Datos/Ejercicio2.xlsx', 'Cold',skiprows = 1,index_col=0)
Cold_df

In [ ]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
s=hotst.copy()
s.insert(0,'S')
w=coldst.copy()
w.insert(0,'W')

model.i=Set(initialize=hotst)
model.j=Set(initialize=coldst)
model.s=Set(initialize=s)
model.S=Set(initialize='S')
model.w=Set(initialize=w)
model.W=Set(initialize='W')
model.k=Set(initialize=Hot_df.columns)


Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [ ]:
model.pprint()

Creamos ahora las variables del modelo. Hay i potenciales aportes de calor por utilities calientes (Qs), uno en cada intervalo, al igual que de frías (Qw). De esta forma damos la oportunidad de que entren las utilities en cualquier nivel de temperatura. Creamos también i residuos (R), aunque sabemos que no habrá residuo del intervalo 0 ni tampoco del intervalo 9, lo cual implementaremos como dos restricciones diferentes.

In [ ]:
Qs = model.Qs = Var(model.k,within = NonNegativeReals) 
Qw = model.Qw = Var(model.k,within = NonNegativeReals) 
R = model.R = Var(model.s,model.k,within = NonNegativeReals)
Q = model.Q = Var(model.s,model.w,model.k,within = NonNegativeReals)

Buscamos la minimización del vapor del intervalo 1 más la utility fría del intervalo 9.

In [ ]:
model.util = Objective(expr = model.Qs[1] + model.Qw[5])

Constraints

In [ ]:
nk = list(model.k)[1:] #Creamos una lista que contenga los índices sobre los que haremos el balance. En nuestro caso el balance lo aplicamos en todos los intervalos, excepto el intervalo 0. Es decir sobre 1-5, lo cual expresamos por [1:], es decir, desde 1 hasta el final de la lista.
model.C1 = ConstraintList()
for k in nk:
    for i in model.i:
        model.C1.add(
        R[i,k-1]+Hot_df.loc[i,k] == R[i,k]+sum(Q[i,j,k] for j in model.j)+sum(Q[i,w,k] for w in model.W)
        )

model.C2 = ConstraintList()
for k in nk:
    for s in model.S:
        model.C2.add(
        R[s,k-1]+Qs[k] == R[s,k]+sum(Q[s,j,k] for j in model.j)
        )        

model.C3 = ConstraintList()
for k in nk:
    for j in model.j:
        model.C3.add(
        sum(Q[i,j,k] for i in model.i)+ sum(Q[i,j,k] for i in model.S) == Cold_df.loc[j,k]
        )

model.C4 = ConstraintList()
for k in nk:
        model.C4.add(
        sum(Q[i,'W',k] for i in model.i)  == Qw[k]
        )


model.C5 = ConstraintList()
for s in model.s:
    model.C5.add(
        R[s,0]==0
        )

model.C6 = ConstraintList()
for s in model.s:
    model.C6.add(
        R[s,5]==0
        )

nm = list(model.k)[0:1]+list(model.k)[2:]
model.C7 = ConstraintList()
for m in nm:
    model.C7.add(
        Qs[m]==0
        )

nn = list(model.k)[0:5]
model.C8 = ConstraintList()
for n in nn:
    model.C8.add(
        Qw[n]==0
        )

model.C9 = ConstraintList()
for k in model.k:
    model.C9.add(
        Q['S','W',k] ==0
    )

Resolvemos el modelo

In [ ]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

In [ ]:
Qss = value(model.Qs[1])
Qww = value(model.Qw[5])
print('Cold utility = {0:2.2f}, Hot utility = {1:2.2f}'.format(Qww, Qss))

Ahora resolvemos el problema de mínimo número de intercambios

In [ ]:
U = {'C1': [60, 280,360], 'C2': [60, 195,195], 'W':[0,225,225]}
U_df = pd.DataFrame(U, index=['S', 'H1', 'H2'])
U_df

In [ ]:
model.y=Var(model.s,model.w, within=Binary)

model.C10 = ConstraintList()
for s in model.s:
    for w in model.w:
        model.C10.add(
        sum(Q[s,w,k] for k in model.k)-U_df.loc[s,w]*model.y[s,w] <=0
        )    

In [ ]:
model.util.deactivate()
model.sumy = Objective(expr=sum(model.y[s,w] for s in model.s for w in model.w))

In [ ]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()